In [1]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import pandas as pd
from sklearn.preprocessing import StandardScaler,MinMaxScaler,OneHotEncoder,OrdinalEncoder
from sklearn.model_selection import GridSearchCV,cross_val_score,train_test_split
import numpy as np

In [2]:
df=pd.read_csv('penguins_size.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   island             344 non-null    object 
 2   culmen_length_mm   342 non-null    float64
 3   culmen_depth_mm    342 non-null    float64
 4   flipper_length_mm  342 non-null    float64
 5   body_mass_g        342 non-null    float64
 6   sex                334 non-null    object 
dtypes: float64(4), object(3)
memory usage: 18.9+ KB


In [4]:
X=df.drop('species',axis=1)
y=df['species']

In [5]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [6]:
numcols=['culmen_length_mm','culmen_depth_mm','flipper_length_mm','body_mass_g']
catcols=['island','sex']

In [7]:
imp=ColumnTransformer([('num',SimpleImputer(missing_values=np.nan,strategy='median'),numcols),
                        ('cat',SimpleImputer(missing_values=np.nan,strategy='most_frequent'),catcols)
                    ])

In [8]:
imp.fit_transform(X_train, y_train)

array([[42.3, 21.2, 191.0, 4150.0, 'Dream', 'MALE'],
       [47.5, 15.0, 218.0, 4950.0, 'Biscoe', 'FEMALE'],
       [46.8, 16.1, 215.0, 5500.0, 'Biscoe', 'MALE'],
       ...,
       [37.3, 20.5, 199.0, 3775.0, 'Torgersen', 'MALE'],
       [37.5, 18.9, 179.0, 2975.0, 'Dream', 'MALE'],
       [42.4, 17.3, 181.0, 3600.0, 'Dream', 'FEMALE']], dtype=object)

In [9]:
pre=ColumnTransformer([('numcol',MinMaxScaler(),numcols),
                        ('catcol',OneHotEncoder(),catcols)
                    ])

In [41]:
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.svm import SVC
class ColumnExtractor(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def transform(self, X, *_):
        return pd.DataFrame(X, columns=self.columns)

    def fit(self, *_):
        return self

pipe20=Pipeline([('impute',imp),
                ('ce', ColumnExtractor(numcols))])

pipe12=Pipeline([('impute',imp),
                ('ce', ColumnExtractor(numcols+catcols)),
                ('preprocessing', pre),
                ('model1', SVC())])

In [42]:
pipe12.fit(X_train, y_train)

Pipeline(steps=[('impute',
                 ColumnTransformer(transformers=[('num',
                                                  SimpleImputer(strategy='median'),
                                                  ['culmen_length_mm',
                                                   'culmen_depth_mm',
                                                   'flipper_length_mm',
                                                   'body_mass_g']),
                                                 ('cat',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  ['island', 'sex'])])),
                ('ce',
                 ColumnExtractor(columns=['culmen_length_mm', 'culmen_depth_mm',
                                          'flipper_length_mm', 'body_mass_g',
                                          'island', 'sex'])),
                ('preprocessing',
                 ColumnTransformer(transformers=[('numcol', MinMaxScaler(),
                                                  ['culmen_length_mm',
                                                   'culmen_depth_mm',
                                                   'flipper_length_mm',
                                                   'body_mass_g']),
                                                 ('catcol', OneHotEncoder(),
                                                  ['island', 'sex'])])),
                ('model1', SVC())])

In [43]:
y_train.size

275

In [44]:
y_pred_train = pipe12.predict(X_train)
len(y_pred_train)

275

In [45]:
from sklearn.metrics import classification_report


print(classification_report(y_train, y_pred_train))

              precision    recall  f1-score   support

      Adelie       0.98      1.00      0.99       118
   Chinstrap       1.00      0.98      0.99        58
      Gentoo       1.00      0.99      0.99        99

    accuracy                           0.99       275
   macro avg       0.99      0.99      0.99       275
weighted avg       0.99      0.99      0.99       275



In [46]:
numcols+catcols

['culmen_length_mm',
 'culmen_depth_mm',
 'flipper_length_mm',
 'body_mass_g',
 'island',
 'sex']

In [47]:
from imblearn.over_sampling import SMOTE

In [48]:
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator, TransformerMixin
class ColumnExtractor(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def transform(self, X, *_):
        return pd.DataFrame(X, columns=self.columns)

    def fit(self, *_):
        return self

pipe=Pipeline([('impute',imp),
                ('ce', ColumnExtractor(numcols+catcols)),
                ('preprocess',pre),
                ('model',LogisticRegression())
                ])

In [49]:
ppipe=Pipeline([('impute',imp),
                ('ce', ColumnExtractor(numcols+catcols)),
                ('preprocess',pre),
                ('smot',SMOTE()),
                ('model',LogisticRegression())
                ])

In [50]:
imp.fit(X_train)

ColumnTransformer(transformers=[('num', SimpleImputer(strategy='median'),
                                 ['culmen_length_mm', 'culmen_depth_mm',
                                  'flipper_length_mm', 'body_mass_g']),
                                ('cat', SimpleImputer(strategy='most_frequent'),
                                 ['island', 'sex'])])

In [51]:
X_train=imp.transform(X_train)

In [52]:
X_test=imp.transform(X_test)

In [53]:
pipe.fit(X_train,y_train)

ValueError: Specifying the columns using strings is only supported for pandas DataFrames

In [ ]:
y_pred_train1=pipe.predict(X_train)

In [ ]:
y_pred_train1

array(['Adelie', 'Gentoo', 'Gentoo', 'Adelie', 'Chinstrap', 'Gentoo',
       'Adelie', 'Adelie', 'Gentoo', 'Chinstrap', 'Chinstrap', 'Adelie',
       'Chinstrap', 'Adelie', 'Adelie', 'Adelie', 'Gentoo', 'Adelie',
       'Gentoo', 'Adelie', 'Adelie', 'Gentoo', 'Gentoo', 'Adelie',
       'Chinstrap', 'Gentoo', 'Gentoo', 'Chinstrap', 'Gentoo', 'Adelie',
       'Adelie', 'Adelie', 'Adelie', 'Adelie', 'Gentoo', 'Chinstrap',
       'Gentoo', 'Gentoo', 'Gentoo', 'Chinstrap', 'Gentoo', 'Gentoo',
       'Adelie', 'Adelie', 'Adelie', 'Adelie', 'Adelie', 'Adelie',
       'Chinstrap', 'Adelie', 'Adelie', 'Chinstrap', 'Adelie',
       'Chinstrap', 'Adelie', 'Gentoo', 'Gentoo', 'Gentoo', 'Chinstrap',
       'Gentoo', 'Adelie', 'Adelie', 'Chinstrap', 'Adelie', 'Gentoo',
       'Gentoo', 'Adelie', 'Adelie', 'Gentoo', 'Gentoo', 'Adelie',
       'Chinstrap', 'Chinstrap', 'Gentoo', 'Gentoo', 'Chinstrap',
       'Chinstrap', 'Chinstrap', 'Gentoo', 'Adelie', 'Gentoo',
       'Chinstrap', 'Chinstrap', 'Chin

In [ ]:
y_pred_train=pipe.predict(X_train)
y_pred_test=pipe.predict(X_test)

NameError: name 'pipe' is not defined

In [ ]:
from sklearn.metrics import classification_report


print(classification_report(y_train,y_pred_train))

              precision    recall  f1-score   support

      Adelie       1.00      1.00      1.00       109
   Chinstrap       1.00      1.00      1.00        59
      Gentoo       1.00      1.00      1.00        99

    accuracy                           1.00       267
   macro avg       1.00      1.00      1.00       267
weighted avg       1.00      1.00      1.00       267

